In [3]:
from groq import Groq
import base64
import os
import dotenv

dotenv.load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [14]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "./Report_Images/Complete-blood-count-CBC-results.png"

# Getting the base64 string
base64_image = encode_image(image_path)

client = Groq()
client.api_key = GROQ_API_KEY

In [35]:



chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", 
                "text": """provide me the details in the image in the form of table, dont type anything else in the response.  
                Make sure that correct details are provided in the table and the alignment is also according to the image.
                just provide me the table and nothing else, not even any disclaimer or note.
                provide the response in the markdown format."""},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ],
    model="llama-3.2-90b-vision-preview",
)



In [36]:
response = chat_completion.choices[0].message.content

In [37]:
print(response)

| Test name | Initial CBC | 18 months later | Unit | Reference range |
| --- | --- | --- | --- | --- |
| White blood cells (WBC) | 1.90 | 4.25 | K/μL | 4.5–11.5 |
| Red blood cells (RBC) | 3.75 | 2.47 | M/μL | 4–5.40 |
| Hemoglobin (Hb) | 11 | 8.8 | g/dL | 12 15 |
| Hematocrit (Hct) | 30.6 | 26.9 | % | 35-49 |
| Mean cell volume (MCV) | 81.6 | 108.9 | fL | 80–94 |
| Mean cell hemoglobin (MCH) | 29.3 | 35.6 | pg | 32–36 |
| Platelets | 12 | 51 | K/μL | 150–450 |
| Reticulocyte absolute | 0.020 | 0.128 | % | 0.5–2 |

**Note:** The table provided does not include all the details from the image, as some information is not easily translatable into a markdown format. The table focuses on key medical test results and their associated units and reference ranges.
